In [84]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize

In [113]:
np.random.seed(2023)


In [110]:
print(U)
print(D)
print(Vt)

[[-0.70710678  0.70710678]
 [ 0.70710678  0.70710678]]
[[5.53957773e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 8.88178420e-16 0.00000000e+00]]
[[-0.24941099 -0.95617472 -0.15337557]
 [-0.78457054  0.10667465  0.61079423]
 [ 0.56766472 -0.27267275  0.77679234]]


In [118]:
# Pre step 1: get and center data

X = np.random.random((100,7)) * 10
mean_subtr = np.mean(X, axis = 0)

# Important, center data
X = X - mean_subtr
X

array([[-1.94547863e+00,  4.04665496e+00,  8.89443337e-01,
        -3.93466897e+00, -4.04815623e+00,  1.77900617e-01,
        -4.93992419e+00],
       [ 2.10738545e+00,  3.86303863e-01,  4.58273171e-01,
        -6.36897305e-01, -4.47745805e-01, -5.56369777e-01,
        -3.64909778e+00],
       [-1.55660990e+00, -3.23679949e+00, -1.61149229e+00,
        -3.39739710e+00, -1.55165444e+00, -4.14457320e+00,
         4.87795678e-01],
       [-3.13074675e+00, -1.65152498e+00, -1.22544139e+00,
        -3.36008849e+00, -4.42205009e+00,  4.82169400e-02,
        -3.20218244e+00],
       [-1.38010745e+00,  4.44775003e+00,  2.61051791e+00,
         2.50701247e+00,  5.05437121e-01,  3.41515617e+00,
         2.94256223e+00],
       [ 4.64021058e+00,  3.99028298e+00, -3.89306796e+00,
         2.99647770e+00, -2.38543955e+00, -1.88610864e+00,
        -1.10358537e+00],
       [ 3.68842145e-01,  1.39769485e+00, -4.20347675e+00,
         4.52220436e+00, -1.34845791e+00,  2.71558873e+00,
         1.4720539

Algorithm 1. General SPCA algorithm
1. let A start at V[,1:k], the loadings of the first k ordinary principal components
2. given A = [$\alpha_1, \cdots, \alpha_k$] solve


In [119]:
# Step 1: Get first k loadings of ordinary principal components
U, s, Vt = np.linalg.svd(X, full_matrices=True)
D = np.zeros((U.shape[0], Vt.shape[0]))
D[:s.size, :s.size] = np.diag(s)



In [120]:
# Step 2: solve elastic net problem for j in 1 ... k
def objective_elastic_net(beta, alpha, X, l, l2):
    assert len(beta) == len(alpha), "shape of principal comp. vector alpha does not coincide with beta"
    return (alpha - beta).T @ X.T @ X @ (alpha - beta) + l * np.linalg.norm(beta) ** 2 + l_2 * np.linalg.norm(beta)

